In [2]:
from ultralytics import YOLO
import cv2
import math
import pytesseract


model_person_vehicle = YOLO("yolo11n.pt")

model_license_plate = YOLO("yolov11-plates/weights/yolov11-plates-last.pt")

classNames = ["person", "bicycle", "car"]

#pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'


cap = cv2.VideoCapture("C0142.MP4")
#cap = cv2.VideoCapture("CochesC.mp4")

output_video_path = "detection_result.mp4"

# Configuración del VideoWriter para volcar el vídeo procesado a disco
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Inicializa el VideoWriter con el codec y la configuración deseada
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))



# Contadores de ID para cada clase
person_counter = 0
car_counter = 0

# Diccionario para asignar IDs únicos según el objeto (usando su track_id)
tracked_ids = {}
fotograma = 0

def write_line(result):
    csv_line = ""
    for i in range(0, len(result)-2):
        csv_line += str(result[i]) + ","
    csv_line += result[len(result)-2] + result[len(result)-1]
    
    nombre_archivo = "results.csv"
    with open(nombre_archivo, 'a') as archivo:
        archivo.write(csv_line)


while (cap.isOpened()):
    ret, frame = cap.read()
    fotograma += 1

    if not ret:
        break

    results_person_vehicle = model_person_vehicle.track(frame, persist=True, classes = [0,1,2])
    
    # Iterar sobre cada detección
    for result in results_person_vehicle:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])  # Clase de la detección
            track_id = int(box.id[0]) if box.id is not None else -1

            # Comprobar si el ID de seguimiento ya está registrado
            if track_id not in tracked_ids:
                # Asignar un nuevo ID basado en la clase del objeto
                if cls == 0:  # Persona
                    person_counter += 1
                    tracked_ids[track_id] = f"P{person_counter}"
                elif cls == 2:  # Coche
                    car_counter += 1
                    tracked_ids[track_id] = f"C{car_counter}"
                else:
                    # Si es una clase diferente, no incrementar contadores
                    tracked_ids[track_id] = f"{track_id}"

            # Obtener el prefijo con el ID específico de la clase
            prefixed_id = tracked_ids[track_id]

            x1, y1, x2, y2, mx1, my1, mx2, my2, xp1, yp1, xp2, yp2 = 0,0,0,0,0,0,0,0,0,0,0,0
            avg_confidence = 0
            license_plate_text = ""
            matricula = "No"

            
            if cls == 0 or cls == 2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
            
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100

                print("Confianza --->",confidence)

                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255 * 2:
                    R, G, B = 255, 255, escala - 255 * 2
                elif escala >= 255:
                    R, G, B = 255, escala - 255, 0
                else:
                    R, G, B = escala, 0, 0



                # Dibujar el contenedor
                cv2.rectangle(frame, (x1, y1), (x2, y2), (R, G, B), 1)
                # Ponemos el texto con la clase, el ID y la confianza
                cv2.putText(frame, f"{classNames[cls]} ID:{prefixed_id} {confidence}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (R, G, B), 2)
                
                if cls == 0:
                    # Censurar el bounding box de la persona con un blur
                    roi = frame[y1:y2, x1:x2]
                    roi = cv2.GaussianBlur(roi, (25, 25), 0)
                    frame[y1:y2, x1:x2] = roi

                    result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                    write_line(result)

                elif cls == 2:
                    # Detección de matrículas
                    results_license_plate = model_license_plate(frame[y1:y2, x1:x2])

                    # Si no se detecta ninguna matrícula escribe directamente en el archivo
                    if(len(results_license_plate) == 0):
                        result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                        write_line(result)

                    # Si se detecta una matrícula
                    for result in results_license_plate:
                        boxes = result.boxes
                        for box in boxes:

                            # Coordenadas de la matrícula en el bounding box
                            xp1, yp1, xp2, yp2 = map(int, box.xyxy[0])

                            # Coordenadas de la matrícula en el fotograma original
                            mx1 = xp1+x1
                            my1 = yp1+y1
                            mx2 = xp2+x1
                            my2 = yp2+y1
                            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (0, 255, 0), 1)

                            # Si la detección es errónea
                            if mx1 == 0 and my1 == 0 and mx2 == 0 and my2 == 0:
                                matricula = "No"
                                continue

                            # Región de interés
                            license_plate = frame[my1:my2, mx1:mx2]

                            # Intentos de tratamiento de la imagen

                            # Umbralizado de la matrícula
                            gray_plate = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
                            gaussian_plate = cv2.GaussianBlur(gray_plate, (3, 3), 0)
                            _, binary_plate = cv2.threshold(gaussian_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                            # Reescalado x3 de la imagen de la matrícula
                            scale_percent = 3
                            width = int(license_plate.shape[1] * scale_percent)
                            height = int(license_plate.shape[0] * scale_percent)
                            dim = (width, height)
                            license_plate_upscaled = cv2.resize(license_plate, dim, interpolation = cv2.INTER_LINEAR)
                            
                            #Mostrar la imagen de la matrícula reescalada
                            #cv2.imshow("Matricula upscaled", license_plate_upscaled)

                            # Pasar la imagen (no) procesada a pytesseract
                            license_plate_text = pytesseract.image_to_string(license_plate, config=custom_config)
                            
                            #Usar image_to_data para obtener detalles de detección y confianza
                            data = pytesseract.image_to_data(license_plate, config=custom_config, output_type=pytesseract.Output.DICT)

                            #Extraer el texto y calcular la confianza promedio
                            text = " ".join([data['text'][i] for i in range(len(data['text'])) if int(data['conf'][i]) > 0])
                            confidences = [int(data['conf'][i]) for i in range(len(data['conf'])) if int(data['conf'][i]) > 0]

                            #Calcular la confianza promedio
                            if confidences:
                                avg_confidence = sum(confidences) / len(confidences)
                            else:
                                avg_confidence = 0  # Si no hay palabras detectadas

                            print("Texto detectado:", text)
                            print("Confianza promedio:", avg_confidence)

                            print("Matrícula detectada:", license_plate_text)
                            matricula = "Si"
                            license_plate_text = license_plate_text.replace("\n", "")

                            # Dibuja el contenedor y matrícula
                            cv2.putText(frame, license_plate_text, [mx1, my1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            print("Matrícula -->", license_plate_text)

                            # Censurar el bounding box de la matrícula con un blur
                            license_plate = cv2.GaussianBlur(license_plate, (25, 25), 0)
                            frame[my1:my2, mx1:mx2] = license_plate

                    result = (fotograma, classNames[cls], confidence, prefixed_id, x1, y1, x2, y2, matricula, avg_confidence, mx1, my1, mx2, my2, license_plate_text, "\n")
                    write_line(result)

                    
    
    
    # Escribimos frame
    out.write(frame)

    # Mostrar el fotograma procesado (opcional)
    cv2.imshow("Deteccion de vehiculos y matriculas", frame)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 23.0ms
Speed: 3.3ms preprocess, 23.0ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.85
Clase --> car

0: 512x640 (no detections), 19.9ms
Speed: 1.0ms preprocess, 19.9ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 19.8ms
Speed: 2.1ms preprocess, 19.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.82
Clase --> car

0: 512x640 (no detections), 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 13.7ms
Speed: 3.0ms preprocess, 13.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.82
Clase --> car

0: 512x640 (no detections), 13.0ms
Speed: 2.2ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 1 car, 22.2ms
Speed: 3.0ms preprocess, 22.2ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)
Confian